In [1]:
from elasticsearch import Elasticsearch, helpers
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('recipes.parquet')
df.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                           0
RecipeIngredientQuantities         0
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

In [4]:
df[['AggregatedRating','ReviewCount']].sort_values(by='ReviewCount', ascending=False).fillna(0)

,AggregatedRating,ReviewCount
41924,5.0,3063.0
1436,5.0,2273.0
23725,5.0,1692.0
84057,4.5,1657.0
35377,5.0,1586.0
...,...,...
522512,0.0,0.0
522513,0.0,0.0
522514,0.0,0.0
522515,0.0,0.0


In [5]:
cleaned_df = df
cleaned_df['CookTime'] = cleaned_df['CookTime'].fillna('')
cleaned_df['AggregatedRating'] = cleaned_df['AggregatedRating'].fillna(0)
cleaned_df['ReviewCount'] = cleaned_df['ReviewCount'].fillna(0)
cleaned_df['Description'] = cleaned_df['Description'].fillna('')
cleaned_df['RecipeCategory'] = cleaned_df['RecipeCategory'].fillna('')
cleaned_df['RecipeServings'] = cleaned_df['RecipeServings'].fillna(0)
cleaned_df['RecipeYield'] = cleaned_df['RecipeYield'].fillna('0')
for row in cleaned_df.loc[cleaned_df['Images'].isna()].index:
    cleaned_df.at[row, 'Images'] = []
cleaned_df.isna().sum()

RecipeId                      0
Name                          0
AuthorId                      0
AuthorName                    0
CookTime                      0
PrepTime                      0
TotalTime                     0
DatePublished                 0
Description                   0
Images                        0
RecipeCategory                0
Keywords                      0
RecipeIngredientQuantities    0
RecipeIngredientParts         0
AggregatedRating              0
ReviewCount                   0
Calories                      0
FatContent                    0
SaturatedFatContent           0
CholesterolContent            0
SodiumContent                 0
CarbohydrateContent           0
FiberContent                  0
SugarContent                  0
ProteinContent                0
RecipeServings                0
RecipeYield                   0
RecipeInstructions            0
dtype: int64

In [6]:
cleaned_df['RecipeYield'].values

array(['0', '0', '0', ..., '0', '20 rolls', '0'], dtype=object)

In [7]:
cleaned_df.to_parquet('no_na_recipes.parquet.gzip', compression='gzip')

In [11]:
class es_indexer:
    def __init__(self):
        self.df = pd.read_parquet('no_na_recipes.parquet.gzip')
        self.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic","ScW45_K41iKePloVvMat"), ca_certs="~/http_ca.crt")

    def run_indexer(self):
        self.es_client.indices.create(index='foodir', ignore=400)
        self.es_client.indices.delete(index='foodir', ignore=[400, 404])
        self.df['_index'] = 'foodir'
        j = json.loads(self.df.to_json(orient='records'))
        helpers.bulk(self.es_client, j)

In [12]:
es = es_indexer()
es.run_indexer()

/var/folders/fq/1w9qg56n37q1h8dkdmps3n100000gn/T/ipykernel_46431/1385807689.py:7: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.create(index='foodir', ignore=400)
/var/folders/fq/1w9qg56n37q1h8dkdmps3n100000gn/T/ipykernel_46431/1385807689.py:8: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.delete(index='foodir', ignore=[400, 404])
